This excercise demonstrates a method to conflate real-time speed attributes from the National Performance Management Research Dataset to the Colorado DOT's modeling network. In this first chapter, the files necesarry for the conflation will be created.

The next cell is used to import necessary modules and set the directy for the output of files. If the output directory (\\Data) doesn't already exist, this directory will be created.

In [3]:
# this process will use arcpy to determine the sameness of two networks
import arcpy as ap

# the csv module is used at the end of this process to write tables to csv
import csv

# the os module is also used in this process, specifically to determine the project location
import os

# shapefile of the tmc network, from the National Performance Management Research Dataset
tmc_network_file = os.getcwd() + '\\Colorado.shp'
    

# shapefile of the Colorado DOT's modeling network 
dtd_network_file = os.getcwd() + '\\CDOT_Network_2015_HwyLinks.shp'

# the project path is set to be the location of this file
path_of_project = os.getcwd()

# and the directory for data is a directory called Data in the project directory
path_to_data = path_of_project + '//Data//'

# if the directory for the data hasn't been created
if not os.path.exists(path_to_data):
    # it will be created
    os.makedirs(path_to_data)

# arcpy has a workplace parameter that sets the output to the data directory
ap.env.workspace = path_to_data

A primary point of comparison of these models will be the lines' bearing, so in the following cell bearing will be added as an attribute to each network.

In [4]:
# iterate the the list of network files
for each in [tmc_network_file, dtd_network_file]:
    
    # and try to add the bearing attribute
    try:
        ap.AddGeometryAttributes_management(each, 'LINE_BEARING')
    # if not possible, then just pass
    except:
        pass

It's important to consider many points along a line, while potentially overkill, this conflation methodolgy uses points every 50 Meters along each segment. 

In [ ]:
try:
    # generate points every 50 meters along the tmc network, excluding the end points
    ap.GeneratePointsAlongLines_management(tmc_network_file, 
                                       path_to_data + 'tmc_network_points.shp', 
                                       'DISTANCE', 
                                       '50 Meters',
                                       '', 
                                       'NO_END_POINTS' )
except:
    pass

The next step will be to consider the proximity of the points along the TMC segments in relation to the modeling network. The following cell creates a table that describes the closest point on the modeling network to the points on the TMC network.

In [2]:
try:
    # generate a table describing the proximity of points every 50 Meters along tmc network in relation the modeling network
    # this table incudes the three closest points along the modeling network within 75 Meters of the tmc network
    # it includes the tmc point and the closest points along the modeling network as well as the bearing
    # and distance of between the points
    ap.GenerateNearTable_analysis(path_to_data + 'tmc_network_points.shp', 
                              dtd_network_file,
                              path_to_data + 'tmc_points_every_50M_near_dtd_network.dbf',
                              '75 Meters',
                              'LOCATION',
                              'ANGLE',
                              'ALL',
                               3,
                              'GEODESIC')
except:
    pass

The cell above creates a relationship between the points on the tmc network to the modeling network. The following cell relates the tmc points back to the tmc network using the 'ORIG_FID' field that is created when the points are tmc network points are created. This is accomplished using a dictionary, which stores the association between the point and the segment and a couple cursors. The first cursor is used to populate the dictionary, and the second retrieves the dictionary values and updates the TMC near table with the id of the tmc segment.

In [4]:
# create a dictionary object called line_id_dict
line_id_dict = {}

# establish a cursor object to iterate through the tmc network points, returning the fid of the point and the ORIG_FID, 
# ORIG_FID represents the FID of the line segment the point was derived from
with ap.da.SearchCursor(path_to_data + 'tmc_network_points.shp', ['FID', 'ORIG_FID']) as sc:
    for row in sc:
        line_id_dict[row[0]] = row[1]

# add a field to the near table to store the original line segment id
ap.AddField_management(path_to_data + 'tmc_points_every_50M_near_dtd_network.dbf', 'LINE_ID', 'SHORT')

# create a cursor to iterate through the near table and update the original line segment id in the near table
with ap.da.UpdateCursor(path_to_data + 'tmc_points_every_50M_near_dtd_network.dbf',
                       ['IN_FID', 'LINE_ID']) as uc:
    for row in uc:
        new_row = row
        # assign the original segment id based on the values stored in the line_id_dictionary
        new_row[1] = line_id_dict[row[0]]
        uc.updateRow(new_row)

In [5]:
# create a csv file to write the near table to, this will allow the table to be accessed with non-prorietary software
with open(path_to_data + 'matches.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    
    # write the header to the csv file
    writer.writerow(['match_id',
                     'tmc_id', 
                     'dtd_id', 
                     'distance',
                     'NEAR_RANK',
                     'NEAR_ANGLE', 
                     'FROM_X', 
                     'FROM_Y', 
                     'NEAR_X', 
                     'NEAR_Y'])
    
    # establish a search cursor to iterate through matches
    with ap.da.SearchCursor(path_to_data + 'tmc_points_every_50M_near_dtd_network.dbf', 
                             ['OID',
                             'LINE_ID', 
                             'NEAR_FID', 
                             'NEAR_DIST',
                             'NEAR_RANK',
                             'NEAR_ANGLE',
                             'FROM_X',
                             'FROM_Y',
                             'NEAR_X',
                             'NEAR_Y']) as sc:
        # for each row in the cursor
        for row in sc:
            # write the row to the csv file
            writer.writerow(row)

In [4]:
# create a csv file to writer the tmc network attributes to
with open(path_to_data + 'tmc_attributes.csv', 'wb') as csvfile:
    
    # create an object to write to the csvfile
    writer = csv.writer(csvfile)
    
    # write the header row
    writer.writerow(['tmc_id', 'tmc_name', 'tmc_bearing', 'TMC', 'tmc_direction', 'tmc_type'])
    
    # iterate through the tmc network file
    with ap.da.SearchCursor (tmc_network_file, ['FID', 'RoadName', 'BEARING', 'TMC', 'N_or_P', 'F_type']) as sc:
        
        for row in sc:
            # store the attributes in the correct data type
            fid = row[0]
            name = row[1].encode('utf')
            bearing = round(row[2],2)
           
            tmc = row[3]
            direction = row[4]
            tmc_type = row[5]
            
            # write the row to the csv file
            writer.writerow([fid, name, bearing, tmc, direction, tmc_type ])

In [7]:
# create a csv file to writer the modeling network attributes to
with open(path_to_data + 'dtd_attributes.csv', 'wb') as csvfile:
    
    # create an object to write to the csvfile
    writer = csv.writer(csvfile)
    
    # write the header row
    writer.writerow(['dtd_id', 'dtd_name', 'dtd_type','dtd_bearing', 'dtd_one_way'])
    
    # iterate through the modeling network file
    with ap.da.SearchCursor (dtd_network_file, ['FID', 'ROUTE', 'FACILITY_T', 'BEARING', 'DIR']) as sc:
        
        for row in sc:
            
            # store all the attributes in the correct data type
            fid = row[0]
            name = row[1].encode('utf')
    
            fclass = row[2]
            bearing = round(row[3],2)
            
            if row[4] == 1:
                one_way = 'N'
            else:
                one_way = 'Y'
                
            # write the row to the csv file
            writer.writerow([fid, name, fclass, bearing, one_way])

In [8]:
# create a line feature class based on the matches.csv, this can be used to demonstrate all of the 
ap.XYToLine_management(path_to_data + 'matches.csv', 
                         path_to_data + 'tmc_to_dtd_match_lines.shp',
                         'FROM_X',
                         'FROM_Y',
                         'NEAR_X',
                         'NEAR_Y',
                         'GEODESIC',
                         'match_id')

ap.AddGeometryAttributes_management(path_to_data + 'tmc_to_dtd_match_lines.shp', 'LENGTH_GEODESIC')

<Result 'C:\\Users\\mcintyrei\\Desktop\\Definitely\\Here TMC to DTD Conflation//Data//tmc_to_dtd_match_lines.shp'>

The next level of analysis will consider only the divided highways. It's simple to determine these segments based on their attributes, but this is not always accurate. However, this will not be addressed until we are done with thie initial analysis.